In [ ]:
%%HTML
<style>
.container { width:100% }
</style>

# Fazit
Abschließend soll ein Fazit zum entwickelten Mühle-Spiel gezogen werden. Dazu wird die Performance der implementierten Algorithmen Alpha-Beta und Minimax verglichen und beurteilt. Desweiteren soll analysiert werden, mit welchen Maßnahmen die Künstliche Intelligenz weiter verbessert werden könnte.

## Bewertung: Alpha-Beta vs. Minimax
Um die Leistung der beiden Algorithmen zu bewerten, wurden x (TODO) Spiele abgehalten, bei denen Alpha-Beta gegen Minimax spielte. Die Such-Tiefe für Alpha-Beta betrug dabei (TODO), während Minimax nur bis zu einer Tiefe von (TODO) suchte. Diese Werte wurden so gewählt, da (TODO). Die Ergebnisse der einzelnen Spiele sind der folgenden Tabelle zu entnehmen. 
Für die Bewertung der Zugzeiten ist beachten, dass die Spiele auf einem Rechner mit folgender Spezifikation durchgeführt wurden: (TODO)

Spiel Nr. | Gewinner | Ø Zugzeit Alpha-Beta (s) | Ø Zugzeit Minimax (s) | Anz. Zustände im Cache Alpha-Beta (bei Spielende) | Anz. Zustände im Cache Minimax (bei Spielende)
:--:|:--:|--|--|--|--
Beispiel | Unentschieden | 55 | 44 | 987654 | 456789
TODO | TODO | TODO | TODO | TODO | TODO 

Damit ergibt sich:

KI | Anz. Spiele verloren | Anz. Spiele unentschieden | Anz. Spiele gewonnen | Anteil verloren | Anteil unentschieden | Anteil gewonnen
:--|--|--|--|--|--|--
**Alpha-Beta** | TODO | TODO | TODO | TODO % | TODO % | TODO %
**Minimax** | TODO | TODO | TODO | TODO % | TODO % | TODO %

Sowie:

KI | Ø Zugzeit (s) | Ø Anz. Zustände im Cache (bei Spielende) 
:--|--|--
**Alpha-Beta** | TODO | TODO
**Minimax** | TODO | TODO

Daran ist zu sehen, dass (TODO)

Die bessere KI ist damit (TODO), weil (TODO)

Die Stärken/Schwächen/Vorteile/Nachteile der KIs sind (TODO)

## Ausblick

Das entwickelte Mühle-Spiel ist in der Lage, verschiedene Spielmodi anzubieten. Das Spielen gegen eine KI, sowie das Spiel KI gegen KI ist möglich, sodass das grundlegende Ziel dieser Arbeit erfüllt wurde. Auch die Verwendung von Transpositionstabellen als Speicher für die bereits bewerteten Zustände wurde umgesetzt. Eine Bewertung der Ergebnisse wurde im vorangehenden Kapitel vorgenommen. Nun soll auf Möglichkeiten eingegangen werden, die die Performance optimieren oder die Stärke der KI steigern.

Die erste Möglichkeit hier ist die Verbesserung der Heuristik. Da die Heuristik die Bewertung der Zustände vornimmt und somit über den nächsten Zustand entscheidet, ist sie der Ansatzpunkt für Leistungssteigerungen. Aktuell umfasst die Heuristik die Aspekte Anzahl der Spielsteine, Anzahl der geschlossenen Mühlen und Anzahl der offenen Mühlen (TODO). Überlegungen für weitere Aspekte sind die Betrachtung der Anzahl der Zugmöglichkeiten und die Bewertung von bestimmten Stellungen. Kritierien für die Entschidung sind  die Veränderung der Rechenzeit und des Gewinnanteils. Ist Erhöhung der Rechenzeit möglichst niedrig, während der Anteil der Gewinne gesteigert werden kann, sollte der Aspekt in die Heuristik mit aufgenommen werden.

Eine weitere Verbesserungsmöglichkeit ist die Einführung einer Spieldatenbank zur Ermittlung besserer Züge. Prinzipiell ist so eine Datenbank für das gesamte Spiel nutzbar, aufgrund des hohen Speicherbedarfs wird häufig jedoch nur eine Endspieldatenbank umgesetzt. Gerade in der dritten Spielphase, wenn die Spieler viele Zugmöglichkeiten haben und der Spielbaum so stark verzweigt ist, kann dies hilfreich sein. Ein Grundstein für die Erstellung und Verwendung einer solchen Spieldatenbank wurde mit dem Abspeichern der Spiele schon gelegt. Eine richtige Verwendung der Datenbank lag jedoch nicht im Anforderungsrahmen dieser Studienarbeit. 

Eine weitere Optimierungsmöglichkeit, um schneller mehr Zustände abzudecken und den Speicherbedarf zu senken ist die Beachtung von Symmetrien und Drehungen. Die Grafik unten veranschaulicht die Symmetrien eines Spielfeldes. Die roten Achsen sind einfache Spiegelungen. Aber auch das Tauschen des inneren und äußeren Rings ist möglich. Dies wird durch die grünen Linien veranschaulicht.

<img src="images\symmetries.png" alt="Symmetrien eines Spielbrettes" width="300"/>

Auch eine Drehung des Spielfelds um 90, 180 oder 270 Grad verändert den eigentlichen Zustand nicht, weshalb auch hier eine Vereinfachung stattfinden kann.

Bei der Überlegung, ob Symmetrien und Drehungen betrachtet werden sollen, ist zu beachten, dass zwar der Speicheraufwand und die Anzahl der möglichen Zustände reduziert werden, die Rechenzeit jedoch stark zunimmt.

Abschließend ist zu sagen, dass noch weitere Verbesserungen möglich sind, dies aber immer unter Abwägung der Vor- und Nachteile und vor allem hinsichtlich der benötigten Rechenzeit geschehen sollte.